
Temujin U. Lampasa
October 8, 2021

I/we certify that this submission complies with the DISCS Academic Integrity Policy.

If I/we have discussed my/our Python language code with anyone other than my/our instructor(s), my/our groupmate(s), the teaching assistant(s), the extent of each discussion has been clearly noted along with a proper citation in the comments of my/our program. If any Python language code or documentation used in my/our program was obtained from another source, either modified or unmodified, such as a textbook, website, or another individual, the extent of its use has been clearly noted along with a proper citation in the comments of my/our program.

# Quiz/Hands-on: Exploring extracted tweets

**Note:** Although the quiz directions indicate to use the previously extracted tweets, I found them to be unsuitable for this activity, as there were not enough data points and it was of a boring topic. Instead, I have rewritten the extract_tweets notebook and scraped 500 tweets from a different topic. The notebook can be found here: https://github.com/temujin-lampasa/social_computing/blob/main/notebooks/extract_tweets.ipynb


In [114]:
!pip install -q pandas numpy
!python -m -q spacy download en_core_web_sm

/Users/TL/.pyenv/versions/3.8.5/bin/python: No module named -q


In [115]:
# Imports
import pandas as pd
import numpy as np
import spacy
from nltk.corpus import stopwords
import re
import os

In [116]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/TL/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [117]:
# Open tweets csv
df = pd.read_csv("../resources/tweets.csv", sep="|")
df

,date,handle,text
0,Fri Oct 08 12:15:32 +0000 2021,DakotaDW_,"RT @Vahn16: if I was ""committed to making acti..."
1,Fri Oct 08 12:15:15 +0000 2021,ACanadianHorse,RT @Kotaku: California steps in to prevent Act...
2,Fri Oct 08 12:15:07 +0000 2021,CHFHope,Which of these flavours is your fave? \nShare ...
3,Fri Oct 08 12:14:46 +0000 2021,ItsOasus,RT @Kotaku: California steps in to prevent Act...
4,Fri Oct 08 12:14:37 +0000 2021,ali18pot,RT @dk_deters: CHRISTMAS ONCE AGAIN\nA missing...
...,...,...,...
495,Fri Oct 08 08:11:48 +0000 2021,Stubborn_Tort,"RT @Vahn16: if I was ""committed to making acti..."
496,Fri Oct 08 08:11:26 +0000 2021,sguedex,"RT @Vahn16: if I was ""committed to making acti..."
497,Fri Oct 08 08:11:18 +0000 2021,y0ungsean,Taking a hobbit through a blizzard with ⁦@vala...
498,Fri Oct 08 08:11:07 +0000 2021,JayromeMarzan,RT @LambySeriesGG: Courtesy of Blizzard i have...


## Prepare and Clean Corpus

In [145]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [146]:
# Get list of stopwords
stop = set(stopwords.words('english'))

# Add interjections like uhh, fuck
stop.update(['fuck', 'fucking'])
stop.update([f"u{'h'*i}" for i in range(10)])

In [238]:
def clean_text(text: str, nlp: spacy.lang.en.English)->str:    
    
    # Remove leading and trailing whitespaces
    text = text.strip()
    
    # Lowercase
    text = text.lower()
    
    # Remove twitter handles
    text = re.sub('@[\w\d]+', '', text)
    
    # Tokenize and Lemmatize
    doc = nlp(text)
    
    # Build new string from processed tokens
    result_string = ""
    
    for token in doc:
        # Remove stopwords
        if token.text in stop:
            continue
        
        word = token.lemma_
    
        # Remove any special characters for each word
        word = re.sub('[^A-Za-z0-9\.]+', '', word)
    
        # Remove consecutive whitespaces
        # (they may have formed after the previous transformations)
        word  = re.sub('\s+', ' ', word)
        
        # Add to resulting string
        if not word:
            continue
        
        if word != "." and len(result_string)!=0:
            result_string += " " + word
        else:
            result_string += word
            
    return result_string
    

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Error: df must be of type pd.DataFrame.")
        
    cleaned_df = df.copy()
        
    # Remove blank tweets
    cleaned_df['text'].replace("", np.nan, inplace=True)
    cleaned_df.dropna(subset=['text'], inplace=True)
    cleaned_df['text'] = df['text'].apply(lambda x: clean_text(x, nlp))
    
    return cleaned_df

In [245]:
# Test
test_text = "      @jeff @jeffrey @jeffreys This is my tweet. There are many like it. But this one is mine. "
clean_text(test_text, nlp)

'tweet. many like. one mine.'

In [244]:
cleaned_df = clean_df(df)
cleaned_df

,date,handle,text
0,Fri Oct 08 12:15:32 +0000 2021,DakotaDW_,rt commit make activision blizzard welcoming i...
1,Fri Oct 08 12:15:15 +0000 2021,ACanadianHorse,rt california step prevent activision blizzard...
2,Fri Oct 08 12:15:07 +0000 2021,CHFHope,flavour fave share blizzard treat friend famil...
3,Fri Oct 08 12:14:46 +0000 2021,ItsOasus,rt california step prevent activision blizzard...
4,Fri Oct 08 12:14:37 +0000 2021,ali18pot,rt christmas miss painting christmas blizzard ...
...,...,...,...
495,Fri Oct 08 08:11:48 +0000 2021,Stubborn_Tort,rt commit make activision blizzard welcoming i...
496,Fri Oct 08 08:11:26 +0000 2021,sguedex,rt commit make activision blizzard welcoming i...
497,Fri Oct 08 08:11:18 +0000 2021,y0ungsean,take hobbit blizzard check eng evernight unsch...
498,Fri Oct 08 08:11:07 +0000 2021,JayromeMarzan,rt courtesy blizzard 3 lich king 3 diablo merc...
